# Create materialized lake views 
1. Use this notebook to create materialized lake views. 
2. Select **Run all** to run the notebook. 
3. When the notebook run is completed, return to your lakehouse and refresh your materialized lake views graph. 


In [3]:
CREATE SCHEMA IF NOT EXISTS gold;

StatementMeta(, e08db073-cfdd-4ad7-bf04-0e51484aaa1f, 4, Finished, Available, Finished, False)

<Spark SQL result set with 0 rows and 0 fields>

In [19]:
CREATE MATERIALIZED LAKE VIEW Shopping_Mart_Gold_LH.gold.Customer_Revenue_Summary_MV
(
    CONSTRAINT valid_customer CHECK (CustomerID IS NOT NULL) ON MISMATCH DROP,
    CONSTRAINT valid_order_amnt CHECK (Total_Amount_Spent>=1000 ) ON MISMATCH DROP
)
COMMENT "Aggregated customer metrics behaviour based on purchases above 1000 Rs."
AS
SELECT 
    c.CustomerID,
    c.CustomerName,
    p.Category,
    c.Location,
    COUNT(o.OrderID) AS Total_Orders,
    ROUND(SUM(o.TotalAmount),2) AS Total_Amount_Spent,
    ROUND(AVG(o.TotalAmount),2) AS Avg_Order_Value,
    MAX(o.OrderDate) AS Last_Order_Date
FROM customers c
JOIN orders o ON c.CustomerID = o.CustomerID
JOIN products p ON p.ProductID = o.ProductID
GROUP BY c.CustomerID, c.CustomerName, p.Category, c.Location;

StatementMeta(, e08db073-cfdd-4ad7-bf04-0e51484aaa1f, 20, Finished, Available, Finished, False)

<Spark SQL result set with 11 rows and 2 fields>

In [1]:
CREATE MATERIALIZED LAKE VIEW Shopping_Mart_Gold_LH.gold.Product_Stock_Analytics_MV
TBLPROPERTIES (delta.enableChangeDataFeed=true)
COMMENT "Pre-computed product metrics with stock status indicators"
AS
SELECT 
    p.ProductID,
    p.ProductName,
    p.Category,
    CAST(p.Stock AS INT) AS Current_Stock_Level,
    CASE 
        WHEN CAST(p.Stock AS INT) < 10 THEN 'Restock Immediately'
        WHEN CAST(p.Stock AS INT) < 50 THEN 'Low Stock'
        ELSE 'Healthy'
    END AS Inventory_Status
FROM products p;

StatementMeta(, cfe0a7d3-3f54-4bf3-a00c-c8a9a1c7fa66, 2, Finished, Available, Finished, False)

<Spark SQL result set with 8 rows and 2 fields>

In [23]:
CREATE MATERIALIZED LAKE VIEW Shopping_Mart_Gold_LH.gold.Product_Ratings_Summary_MV
COMMENT "Pre-computed product review ratings" 
AS
SELECT 
    p.ProductID,
    p.ProductName,
    ROUND(AVG(r.rating),2) AS Average_Rating,
    COUNT(r.rating) AS Total_Ratings,
    SUM(CASE WHEN r.rating >= 4 THEN 1 ELSE NULL END) AS Positive_Ratings,
    SUM(CASE WHEN r.rating <= 3 THEN 1 ELSE NULL END) AS Negative_Ratings
FROM products p
JOIN reviews r ON p.ProductID = r.product_id
GROUP BY p.ProductID, p.ProductName;

StatementMeta(, e08db073-cfdd-4ad7-bf04-0e51484aaa1f, 24, Finished, Available, Finished, False)

<Spark SQL result set with 8 rows and 2 fields>

In [33]:
--DROP MATERIALIZED LAKE VIEW Shopping_Mart_Gold_LH.gold.Sales_Orders_Detail_MV
CREATE MATERIALIZED LAKE VIEW Shopping_Mart_Gold_LH.gold.Sales_Orders_Detail_MV
COMMENT "Detailed view optimized for Drill-Through"
AS
SELECT 
    o.OrderID,
    o.OrderDate,
    c.CustomerName,
    c.Location,
    c.Email,
    p.ProductName,
    p.Category,
    o.Quantity,
    p.UnitPrice,
    o.TotalAmount,
    r.rating AS Review_Rating,
    r.review_text AS Customer_Feedback
FROM orders o
JOIN customers c ON o.CustomerID = c.CustomerID
JOIN products p ON o.ProductID = p.ProductID
LEFT JOIN reviews r ON p.ProductID = r.product_id
AND c.CustomerID = r.customer_id;

StatementMeta(, e08db073-cfdd-4ad7-bf04-0e51484aaa1f, 34, Finished, Available, Finished, False)

<Spark SQL result set with 8 rows and 2 fields>

In [25]:
SELECT * FROM Shopping_Mart_Gold_LH.gold.product_ratings_summary_mv LIMIT 1000

StatementMeta(, e08db073-cfdd-4ad7-bf04-0e51484aaa1f, 26, Finished, Available, Finished, False)

<Spark SQL result set with 50 rows and 6 fields>

In [20]:
SELECT * FROM Shopping_Mart_Gold_LH.gold.customer_revenue_summary_mv LIMIT 1000

StatementMeta(, e08db073-cfdd-4ad7-bf04-0e51484aaa1f, 21, Finished, Available, Finished, False)

<Spark SQL result set with 352 rows and 8 fields>

In [2]:
SELECT * FROM Shopping_Mart_Gold_LH.gold.product_stock_analytics_mv LIMIT 1000

StatementMeta(, cfe0a7d3-3f54-4bf3-a00c-c8a9a1c7fa66, 3, Finished, Available, Finished, False)

<Spark SQL result set with 50 rows and 5 fields>